In [1]:
import requests
import defs
import pandas as pd

In [2]:
# Request has a session object, help maintain persisteng connection
session = requests.Session()

In [3]:
instrument =  "EUR_USD"
count = 10
granularity = "H1"

In [4]:
url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"

In [5]:
url

'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles'

In [6]:
params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
)

In [7]:
params

{'count': 10, 'granularity': 'H1', 'price': 'MBA'}

In [8]:
# To make our request to Onanda Api using get
response = session.get(url, params=params, headers= defs.SECURE_HEADER)

In [9]:
response.status_code

200

In [11]:
data= response.json()

In [12]:
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [14]:
len(data['candles'])

10

In [17]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [18]:
for price in prices:
    for oh in ohlc:
        print(f"{price}_{oh}")


mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [25]:
data['candles'][0]['bid']['o']

{'o': '1.06423', 'h': '1.06521', 'l': '1.06402', 'c': '1.06459'}

In [27]:
our_data= []
for candle in data['candles']:
    if candle['complete'] == False:
        continue
    new_dict = {}
    new_dict['time'] = candle['time']
    new_dict['volume'] = candle['volume']
    #print(new_dict)
    for price in prices:
        for oh in ohlc:
            new_dict[f"{price}_{oh}"]= candle[price][oh]

    our_data.append(new_dict)
print(our_data[0])

{'time': '2023-11-03T11:00:00.000000000Z', 'volume': 3016, 'mid_o': '1.06430', 'mid_h': '1.06528', 'mid_l': '1.06409', 'mid_c': '1.06467', 'bid_o': '1.06423', 'bid_h': '1.06521', 'bid_l': '1.06402', 'bid_c': '1.06459', 'ask_o': '1.06438', 'ask_h': '1.06536', 'ask_l': '1.06416', 'ask_c': '1.06475'}


In [28]:
candles_df = pd.DataFrame.from_dict(our_data)

In [29]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-11-03T11:00:00.000000000Z,3016,1.06430,1.06528,1.06409,1.06467,1.06423,1.06521,1.06402,1.06459,1.06438,1.06536,1.06416,1.06475
1,2023-11-03T12:00:00.000000000Z,12095,1.06468,1.07194,1.06455,1.07094,1.06461,1.07187,1.06447,1.07087,1.06475,1.07202,1.06462,1.07102
2,2023-11-03T13:00:00.000000000Z,12529,1.07095,1.07248,1.06961,1.07172,1.07088,1.07238,1.06953,1.07163,1.07102,1.07257,1.06969,1.07180
3,2023-11-03T14:00:00.000000000Z,12965,1.07223,1.07359,1.06936,1.07199,1.07198,1.07345,1.06928,1.07191,1.07248,1.07375,1.06943,1.07207
4,2023-11-03T15:00:00.000000000Z,7011,1.07200,1.07355,1.07154,1.07324,1.07193,1.07347,1.07146,1.07316,1.07207,1.07363,1.07162,1.07331
5,2023-11-03T16:00:00.000000000Z,5952,1.07322,1.07394,1.07254,1.07394,1.07315,1.07387,1.07246,1.07386,1.07329,1.07402,1.07261,1.07402
6,2023-11-03T17:00:00.000000000Z,4548,1.07394,1.07469,1.07306,1.07366,1.07386,1.07461,1.07299,1.07359,1.07401,1.07477,1.07314,1.07373
7,2023-11-03T18:00:00.000000000Z,2894,1.07372,1.07400,1.07326,1.07359,1.07365,1.07392,1.07318,1.07352,1.07379,1.07407,1.07333,1.07366
8,2023-11-03T19:00:00.000000000Z,2393,1.07354,1.07376,1.07224,1.07248,1.07346,1.07369,1.07217,1.07241,1.07361,1.07383,1.07232,1.07255
9,2023-11-03T20:00:00.000000000Z,1224,1.07250,1.07328,1.07250,1.07286,1.07243,1.07320,1.07206,1.07269,1.07257,1.07347,1.07257,1.07303


In [31]:
candles_df.to_pickle("EUR_USD_H1.pkl")